<a href="https://colab.research.google.com/github/componavt/ProJouRu/blob/main/src/1_process_titles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Process Scopus title (e.g. journal) metadata
This file taken from [dhimmel / scopus 1.process-titles.ipynb](https://github.com/dhimmel/scopus/blob/main/1.process-titles.ipynb)

In [17]:
import pathlib
import re

import slugify

import requests
import pandas as pd
from io import BytesIO

#from utilities import format_issn

In [18]:
# Load dataset
# path = pathlib.Path('download/ext_list_December_2024.xlsx')

url = 'https://raw.githubusercontent.com/componavt/ProJouRu/main/download/ext_list_December_2024.xlsx'

response = requests.get(url)
response.raise_for_status()  # Проверяем, что запрос успешен

source_df = pd.read_excel(BytesIO(response.content), sheet_name='Scopus Sources Dec. 2024')
# print(source_df.head(2))

   Sourcerecord ID          Source Title      ISSN     EISSN  \
0      21101170611                  @GRH  20349130  22959149   
1      21100902951  [Hifu] Skin research  00181390       NaN   

  Active or Inactive   Coverage  \
0             Active  2023-2024   
1           Inactive  1959-2001   

  Titles Discontinued by Scopus Due to Quality Issues  \
0                                                NaN    
1                                                NaN    

  Article Language in Source (Three-Letter ISO Language Codes)  \
0                                           ENG; FRE             
1                                                JPN             

  Medline-sourced Title? (See additional details under separate tab.)  \
0                                                NaN                    
1                                                NaN                    

  Open Access Status  ... 2700\nMedicine 2800\nNeuroscience 2900\nNursing  \
0                NaN  ...        

In [19]:
source_df.columns

Index(['Sourcerecord ID', 'Source Title', 'ISSN', 'EISSN',
       'Active or Inactive', 'Coverage',
       'Titles Discontinued by Scopus Due to Quality Issues',
       'Article Language in Source (Three-Letter ISO Language Codes)',
       'Medline-sourced Title? (See additional details under separate tab.)',
       'Open Access Status', 'Articles in Press Included?',
       'Added to List Dec. 2024', 'Source Type', 'Title History Indication',
       'Related Title 1', 'Other Related Title 2', 'Other Related Title 3',
       'Other Related Title 4', 'Publisher',
       'Publisher Imprints Grouped to Main Publisher',
       'All Science Journal Classification Codes (ASJC)',
       'Top level:\n\nLife Sciences', 'Top level:\n\nSocial Sciences',
       'Top level:\n\nPhysical Sciences', 'Top level:\n\nHealth Sciences',
       '1000 \nGeneral', '1100\nAgricultural and Biological Sciences',
       '1200\nArts and Humanities',
       '1300\nBiochemistry, Genetics and Molecular Biology',
    

In [20]:
conf_df = pandas.read_excel(path, sheet_name='All Conf. Proceedings')
conf_df.head(2)

URLError: <urlopen error no host given>

In [ ]:
code_df = pandas.read_excel(path, sheet_name='ASJC classification codes', usecols=["Code", "Description"], skiprows=8).dropna()
code_df.Code = code_df.Code.astype(int)
code_df.head(2)

In [ ]:
# Rename columns on dataframes
renamer = {
    'Sourcerecord ID': 'scopus_id',
    "Source Title (Medline-sourced journals are indicated in Green)": "title_name",
    "Conference Name": "title_name",
    'All Science Classification Codes (ASJC)': 'asjc_codes',
    'ASJC Code': 'asjc_codes',
    "All Science Journal Classification Codes (ASJC)": "asjc_codes",
    'Print-ISSN': 'issn_print',
    'E-ISSN': 'issn_electronic',
    'ISSN': 'issn_unspecified',
}

for df in source_df, conf_df:
    df.rename(columns=renamer, inplace=True)

## Titles

In [ ]:
# Extract title names
rows = list()
for df in source_df, conf_df:
    rows.extend(zip(df.scopus_id, df.title_name))

rows.sort()
title_df = pandas.DataFrame(rows, columns=['scopus_id', 'title_name'])
title_df = title_df.drop_duplicates('scopus_id')

In [ ]:
title_df.head(2)

In [ ]:
len(title_df)

In [ ]:
path = pathlib.Path('data/titles.tsv')
title_df.to_csv(path, index=False, sep='\t')

## ISSN

In [ ]:
# Load linking ISSN map as per https://github.com/dhimmel/scopus/issues/7
path = pathlib.Path('download/20210912.ISSN-to-ISSN-L.txt.gz')
issnl_df = (
    pandas.read_table(path)
    .rename(columns={"ISSN": "issn", "ISSN-L": "linking_issn"})
)
issnl_df.head(2)

In [ ]:
value_vars = {'issn_print', 'issn_electronic', 'issn_unspecified'}
issn_dfs = list()
for df in source_df, conf_df:
    df_value_vars = value_vars.intersection(df.columns)
    df = pandas.melt(df, id_vars='scopus_id', value_vars=df_value_vars, var_name='issn_type', value_name='issn')
    df.dropna(inplace=True)
    df.issn = df.issn.map(format_issn)
    df.issn_type = df.issn_type.map(lambda x: x.split('_')[1])
    issn_dfs.append(df.dropna())
issn_df = (
    pandas.concat(issn_dfs)
    .dropna()
    .drop_duplicates(['scopus_id', 'issn'])
    .merge(issnl_df, how="left")
    .sort_values(['scopus_id', 'issn_type', 'issn', 'linking_issn'])
)
issn_df.head(2)

In [ ]:
len(issn_df), len(issnl_df)

In [ ]:
issn_df.head(3)

In [ ]:
# Scopus ISSNs that don't have an ISSN-L
bad_issn_df = issn_df[issn_df.linking_issn.isna()]
print(len(bad_issn_df))
bad_issn_df.sample(5, random_state=0)

In [ ]:
path = pathlib.Path('data/issn.tsv')
issn_df.to_csv(path, sep='\t', index=False)

## ASJC Codes

All Science Journal Classification (ASJC) Codes

In [ ]:
# Extract ASJC code descriptions
code_df = code_df.rename(columns={'Code': 'asjc_code', 'Description': 'asjc_description'})
code_df = code_df[['asjc_code', 'asjc_description']]
code_df.head(2)

In [ ]:
path = pathlib.Path('data/asjc-codes.tsv')
code_df.to_csv(path, index=False, sep='\t')

In [ ]:
# Extract title to code mapping
rows = list()
for df in source_df, conf_df:
    for scopus_id, codes in zip(df.scopus_id, df.asjc_codes):
        for code in re.split(r'[,;] *', str(codes)):
            code = code.strip()
            try:
                code = int(code)
            except ValueError:
                continue
            rows.append((scopus_id, code))

rows.sort()
title_codes_df = pandas.DataFrame(rows, columns=['scopus_id', 'asjc_code'])
title_codes_df = title_codes_df.drop_duplicates()
title_codes_df.head(2)

In [ ]:
path = pathlib.Path('data/titles-asjc-codes.tsv')
title_codes_df.to_csv(path, index=False, sep='\t')

In [ ]:
# Extract title to subject area mapping
subject_df = title_codes_df.copy()
subject_df.asjc_code = 100 * (subject_df.asjc_code // 100)
subject_df = subject_df.drop_duplicates()
subject_df = subject_df.merge(code_df)
subject_df.asjc_description = subject_df.asjc_description.map(lambda x: re.sub(r'^General ', '', x))
subject_df = subject_df.sort_values(['scopus_id', 'asjc_code'])
subject_df.head(2)

In [ ]:
path = pathlib.Path('data/subject-areas.tsv')
subject_df.to_csv(path, index=False, sep='\t')

## Title attributes

In [ ]:
# Extract title attributes for non-conference-proceedings
renamer = {
    'Open Access status, i.e., registered in DOAJ. Status May 2020\n': 'open_access',
    'Active or Inactive': 'active',
    'Source Type': 'source_type',
    'Publisher imprints grouped to main Publisher': 'main_publisher',
}
attribute_df = source_df.copy()
attribute_df = attribute_df.rename(columns=renamer)
attribute_df = attribute_df[['scopus_id'] + list(renamer.values())]
attribute_df.active = (attribute_df.active == 'Active').astype(int)
attribute_df.open_access = attribute_df.open_access.notnull().astype(int)
attribute_df = attribute_df.sort_values('scopus_id')
# Enforce column order for diff viewing
columns = ['scopus_id', 'active', 'open_access', 'main_publisher', 'source_type']
attribute_df = attribute_df[columns]
attribute_df = attribute_df.drop_duplicates('scopus_id')
attribute_df.head(2)

### Fix publishers

In [ ]:
print(f'Publishers before filtering: {attribute_df.main_publisher.nunique():,}')

# Strip whitespace sourrounding publisher names
attribute_df.main_publisher = attribute_df.main_publisher.str.strip()
print(f'Publishers after whitespace stripping: {attribute_df.main_publisher.nunique():,}')

# Apply manual publisher name patches
path = pathlib.Path('data/publisher-name-patches.tsv')
patch_df = pandas.read_table(path)
publisher_renamer = dict(zip(patch_df.scopus_main_publisher, patch_df.publisher_name))
attribute_df.main_publisher.replace(publisher_renamer, inplace=True)
print(f'Publishers after renaming: {attribute_df.main_publisher.nunique():,}')

In [ ]:
# Create a publsher dataframe
publisher_df = (
    attribute_df
    .groupby('main_publisher')
    .apply(len)
    .rename('n_journals')
    .reset_index()
)
slugger = slugify.Slugify(to_lower=True)
publisher_df['main_publisher_slug'] = publisher_df.main_publisher.map(slugger)

def summarize(df):
    """
    Condense multiple publishers with the same slug
    """
    # Select name variant used for the most number of titles
    df = df.sort_values(['n_journals', 'main_publisher'])
    return {
        'n_journals': sum(df.n_journals),
        'main_publisher': df.main_publisher.iloc[0],
        'name_variants': len(df),
    }

results = (
    publisher_df
    .groupby('main_publisher_slug')
    .apply(summarize)
)
publisher_df = pandas.DataFrame.from_records(results, index=results.index).reset_index()
publisher_df.head(2)

In [ ]:
# How many publishers had name variants (identified using slug reduction)
publisher_df.name_variants.value_counts()

In [ ]:
# Standardize name variants using slug reductionism
slug_to_publisher = dict(zip(publisher_df.main_publisher_slug, publisher_df.main_publisher))
attribute_df.main_publisher = attribute_df.main_publisher.map(slugger, na_action='ignore').map(slug_to_publisher)
print(f'Publishers after slug reductions: {attribute_df.main_publisher.nunique():,}')

In [ ]:
path = pathlib.Path('data/title-attributes.tsv')
attribute_df.to_csv(path, index=False, sep='\t')

path = pathlib.Path('data/publishers.tsv')
publisher_df.to_csv(path, index=False, sep='\t')

In [ ]:
# Top level subject areas
rows = []
tl_cols = list(source_df.columns[source_df.columns.str.startswith('Top level:')])
for i, series in source_df[['scopus_id'] + tl_cols].iterrows():
    scopus_id = series.scopus_id
    for value in series.iloc[1:].dropna():
        rows.append((scopus_id, value))
rows.sort()
top_df = pandas.DataFrame(rows, columns=['scopus_id', 'top_level_subject'])
top_df.head(2)

In [ ]:
path = pathlib.Path('data/title-top-levels.tsv')
top_df.to_csv(path, index=False, sep='\t')